In [11]:
import pandas as pd

from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

In [3]:
SEARCH_PAGE_URL_TEMPLATE = "https://www.sahibinden.com/fiat?pagingOffset={}&pagingSize=50"
search_pages_urls = [SEARCH_PAGE_URL_TEMPLATE.format(number) for number in range(0, 550, 50)]

In [4]:
user_agent = UserAgent(os=["android"], browsers=["chrome"]).random

options = Options()
options.add_argument("--width=1530")
options.add_argument("--height=1050")
options.set_preference("general.useragent.override", user_agent)

In [6]:
search_pages_response = []

for url in search_pages_urls:
  driver = webdriver.Firefox(options=options)
  driver.get(url)
  search_pages_response.append(driver.page_source)
  driver.quit()

In [7]:
advertisement_links = []

for content in search_pages_response:
  soup = BeautifulSoup(content, "html.parser")

  search_page_advertisement_cards = soup.find_all(
      "a", class_="search-classified-link mui-btn")
  advertisement_links.extend(card["href"]
                            for card in search_page_advertisement_cards)

for index, link in enumerate(advertisement_links):
  advertisement_links[index] = "https://www.sahibinden.com" + link

In [8]:
def get_cleaned_details(content):
  soup = BeautifulSoup(content, "html.parser")
  try:
    lines = soup.find_all("ul", class_="classified-info-list")
    if len(lines) == 0:
      return None
  except AttributeError:
    return None
  cleaned_lines = [line.strip() for line in lines[0].text.split("\n") if line.strip()]
  detail = {}

  for index, line in enumerate(cleaned_lines[:-1]):
    if index % 2 == 0:
      detail[line] = cleaned_lines[index + 1]
  
  return detail

In [ ]:
advertisement_details = []
for link in advertisement_links:
  driver = webdriver.Firefox(options=options)
  driver.get(link)
  details = get_cleaned_details(driver.page_source)
  if details:
    advertisement_details.append(details)
    print(details)
  driver.quit()

In [ ]:
pd.DataFrame(advertisement_details).to_excel("fiat_cars.xlsx", index=False)